In [72]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import pandas as pd
from geopy.geocoders import ArcGIS
from geopy import distance
import folium

locations = pd.read_excel(r'c:\Users\Andre\Clean Data Frame.xlsx')

geolocator = ArcGIS(timeout = 100)

# Add comlumns for geocode, latitude, and longitude
locations["Geocode"] = locations.Address.apply(geolocator.geocode)
locations["Latitude"] = [g.latitude for g in locations.Geocode]
locations["Longitude"] = [g.longitude for g in locations.Geocode]

total_distances = []

for i in range(1,6):
    # Find distance from housing location
    distance_to_housing = distance.distance((locations.loc[0].at["Latitude"], locations.loc[0].at["Longitude"]), (locations.loc[i].at["Latitude"], locations.loc[i].at["Longitude"])).miles
    
    # Find distance from conference location
    distance_to_conference = distance.distance((locations.loc[6].at["Latitude"], locations.loc[6].at["Longitude"]), (locations.loc[i].at["Latitude"], locations.loc[i].at["Longitude"])).miles
    
    # Find distance from each of event locations
    distance_to_events = 0
    for j in range(7,12):
        distance_to_events = distance_to_events + distance.distance((locations.loc[i].at["Latitude"], locations.loc[i].at["Longitude"]), (locations.loc[j].at["Latitude"], locations.loc[j].at["Longitude"])).miles

    # Find distance from each of restuarant locations
    distance_to_restaurants = 0
    for k in range(12,22):
        distance_to_restaurants = distance_to_restaurants + distance.distance((locations.loc[i].at["Latitude"], locations.loc[i].at["Longitude"]), (locations.loc[k].at["Latitude"], locations.loc[k].at["Longitude"])).miles
    
    # Find total distance
    total_distance = distance_to_housing + distance_to_conference + distance_to_events + distance_to_restaurants
    
    total_distances.append(total_distance)
    
min_distance = min(total_distances)

map = folium.Map(location = (39.9,-86.2), zoom_start = 11)

for index in range(22):
    # Mark potential coworking spaces in green
    if (index > 0 and index < 6):
        folium.Marker(location = (locations.loc[index].at["Latitude"], locations.loc[index].at["Longitude"]), icon = folium.Icon(color = 'green')).add_to(map)
    # Mark everything else as default (blue)
    else:
        folium.Marker(location = (locations.loc[index].at["Latitude"], locations.loc[index].at["Longitude"])).add_to(map)
    
index = 1
for dist in total_distances:
    if dist == min_distance:
        best_location = locations.loc[index]
        # Mark most convenient coworking location in red
        folium.Marker(location = (locations.loc[index].at["Latitude"], locations.loc[index].at["Longitude"]), icon = folium.Icon(color = 'red')).add_to(map)
        break
    else:
        index = index + 1
    
print("The most convenient coworking space is " + best_location.at["Name"] + " at " + best_location.at["Address"])
print("")
print("All restaurants, events, and conference locations are shown in blue. Possible coworking locations are shown in green. The most convenient coworking space is shown in red.")
    
display(map)

The most convenient coworking space is The Speak Easy at 5255 Winthrop Ave #110, Indianapolis, IN 46220

All restaurants, events, and conference locations are shown in blue. Possible coworking locations are shown in green. The most convenient coworking space is shown in red.
